In [2]:
import os
import pandas as pd
import numpy as np

In [127]:
os.listdir()

['equity_value_data.csv',
 'features_data.csv',
 '.ipynb_checkpoints',
 'Untitled.ipynb']

In [387]:
features = pd.read_csv('features_data.csv')
df = pd.read_csv('equity_value_data.csv')

In [388]:
df.head()

,timestamp,close_equity,user_id
0,2016-11-16T00:00:00Z,48.16,bcef4fa9b0bdf22bcf7deae708decf03
1,2016-11-17T00:00:00Z,48.16,bcef4fa9b0bdf22bcf7deae708decf03
2,2016-11-18T00:00:00Z,48.16,bcef4fa9b0bdf22bcf7deae708decf03
3,2016-11-21T00:00:00Z,48.16,bcef4fa9b0bdf22bcf7deae708decf03
4,2016-11-22T00:00:00Z,48.16,bcef4fa9b0bdf22bcf7deae708decf03


#### a) percentage of churned users

In [441]:
import datetime

In [394]:
# convert timestamp column from string to datetime
df['timestamp'] = df['timestamp'].apply(lambda x: x[:10])
df['timestamp'] = pd.to_datetime(df['timestamp']).dt.date

In [395]:
# sort by user_id, timestamp, ascending
df = df.sort_values(by=['user_id', 'timestamp'])

In [412]:
# create another timestamp column with lag=1
df['timestamp_lag'] = df.groupby(['user_id'])['timestamp'].shift(-1)

In [448]:
# find the difference of days between two back-to-back timestamps with equity >= 10
df['timestamp_diff'] = df['timestamp_lag'] - df['timestamp'] - pd.Timedelta(days=1)
df['timestamp_diff'][pd.isna(df['timestamp_diff'])] = datetime.date(2017,8,18)-df['timestamp'][pd.isna(df['timestamp_diff'])]
df['timestamp_diff'] = df['timestamp_diff'].dt.days

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [457]:
# find the max timestamp difference for each user
df_q1 = df.groupby(['user_id'])['timestamp_diff'].max().reset_index()

In [458]:
df_q1['ifchurn'] = df_q1['timestamp_diff'].apply(lambda x: 1 if x>=28 else 0)

In [459]:
df_q1.head()

,user_id,timestamp_diff,ifchurn
0,0012db34aa7b083f5714e7831195e54d,3,0
1,001d6c77dbdb3213cead7673f250bfdc,3,0
2,002e4653171ddc61c3cd30603cd7bd3e,3,0
3,00384fa9be6fdca1b786bae70d78f88f,3,0
4,0042aac295a0d4df88f4b83012778bd4,3,0


In [460]:
df_q1['ifchurn'].value_counts()

0    4579
1    1005
Name: ifchurn, dtype: int64

In [590]:
print('percentage of churned users is {}'.format(1005/(1005+4579)))

percentage of churned users is 0.17997851002865328


#### b) build a classifier with random forest

In [544]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, accuracy_score

In [465]:
df = pd.merge(features, df_q1, on='user_id')

In [466]:
# encode categorical features
df['risk_tolerance'] = df['risk_tolerance'].map({'low_risk_tolerance':0,
                                                 'med_risk_tolerance':1,
                                                 'high_risk_tolerance':2})

df['investment_experience'] = df['investment_experience'].map({'no_investment_exp':0,
                                                               'limited_investment_exp':1,
                                                               'good_investment_exp':2,
                                                               'extensive_investment_exp':3})

df['liquidity_needs'] = df['liquidity_needs'].map({'not_important_liq_need':0,
                                                   'somewhat_important_liq_need':1,
                                                   'very_important_liq_need':2})

df['time_horizon'] = df['time_horizon'].map({'short_time_horizon':0,
                                             'med_time_horizon':1,
                                             'long_time_horizon':2})

In [467]:
df = pd.concat([df, 
                pd.get_dummies(df['platform']), 
                pd.get_dummies(df['instrument_type_first_traded'])], 
               axis=1)

In [469]:
df.columns

Index(['risk_tolerance', 'investment_experience', 'liquidity_needs',
       'platform', 'time_spent', 'instrument_type_first_traded',
       'first_deposit_amount', 'time_horizon', 'user_id', 'timestamp_diff',
       'ifchurn', 'Android', 'both', 'iOS', '0', 'adr', 'cef', 'etp', 'lp',
       'mlp', 'reit', 'rlt', 'stock', 'tracking', 'wrt'],
      dtype='object')

In [470]:
X = df.drop(['platform','instrument_type_first_traded','user_id','timestamp_diff', 'ifchurn'], axis=1)
y = df['ifchurn']

In [552]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [704]:
clf = RandomForestClassifier(n_estimators=50, max_depth=10, class_weight={0:1,1:5})
# clf = RandomForestClassifier(n_estimators=50, max_depth=10, class_weight='balanced')

In [705]:
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight={0: 1, 1: 5},
            criterion='gini', max_depth=10, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=50, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [706]:
cutoff = 0.5
y_train_pred = (clf.predict_proba(X_train)[:,1]>=cutoff).astype(int)
y_test_pred = (clf.predict_proba(X_test)[:,1]>=cutoff).astype(int)

In [707]:
print('precision_train:{} \n'.format(precision_score(y_train, y_train_pred)), 
      'recall_train:{} \n'.format(recall_score(y_train, y_train_pred)), 
      'f1_train:{}'.format(f1_score(y_train, y_train_pred)))

print('-'*40)

print('precision_test:{} \n'.format(precision_score(y_test, y_test_pred)), 
      'recall_test:{} \n'.format(recall_score(y_test, y_test_pred)), 
      'f1_test:{}'.format(f1_score(y_test, y_test_pred)))

precision_train:0.4059065934065934 
 recall_train:0.7838196286472149 
 f1_train:0.534841628959276
----------------------------------------
precision_test:0.3139784946236559 
 recall_test:0.5816733067729084 
 f1_test:0.4078212290502793


* Because the outcome label classes are imbalanced, the metrics I used here are precision, recall and f1 score.
* I did random train test split on the data, keeping the same 0-1 label ratio in both train and test datasets
* A random forest classifier is fitted, I restricted the number of trees and max depth of each tree, and gave more weights to samples with outcome = 1

#### c) feature importance

In [659]:
feature_importance = pd.DataFrame({'features': X_train.columns,
                                   'importance': clf.feature_importances_})

In [709]:
feature_importance.sort_values(['importance'], ascending=False)

,features,importance
3,time_spent,0.313644
4,first_deposit_amount,0.279828
0,risk_tolerance,0.084087
1,investment_experience,0.065546
5,time_horizon,0.058652
2,liquidity_needs,0.047177
7,both,0.021136
6,Android,0.020359
8,iOS,0.019676
10,adr,0.018675


* the most important features are time_spent and first_deposit_amount

In [551]:
corr = pd.concat([y,X],axis=1).corr()
corr.style.background_gradient(cmap='coolwarm')

,ifchurn,risk_tolerance,investment_experience,liquidity_needs,time_spent,first_deposit_amount,time_horizon,Android,both,iOS,0,adr,cef,etp,lp,mlp,reit,rlt,stock,tracking,wrt
ifchurn,1,-0.125477,-0.0420861,-0.0661286,0.0325377,-0.0684815,-0.0494142,0.0322114,-0.0111972,-0.0231748,-0.0129584,-0.021366,-0.00467852,-0.0330718,0.0192284,0.00519794,-0.00424272,0.00441812,0.0330134,-0.00626995,0.0184916
risk_tolerance,-0.125477,1,0.142245,0.0333715,-0.0114075,0.0159824,0.152741,0.00633151,-0.0151734,0.00317631,-0.00484878,0.00446089,0.0109335,0.0233206,0.0102224,0.0134275,0.0197755,0.0127969,-0.0283593,-0.0139567,-0.0265816
investment_experience,-0.0420861,0.142245,1,0.0531625,0.0181259,0.127102,0.124193,0.0290484,0.0282009,-0.043725,0.0009213,0.0128387,0.0270864,0.0956371,0.0171125,0.0144571,0.0168573,-0.00105184,-0.0901099,0.00161787,-0.0201267
liquidity_needs,-0.0661286,0.0333715,0.0531625,1,-0.0119651,0.0168379,0.011693,0.290207,0.0593602,-0.304295,-0.028669,0.0130166,-0.0284096,0.00318489,0.00284207,0.00333626,0.0132898,0.00505674,-0.000281965,-0.0173607,-0.032757
time_spent,0.0325377,-0.0114075,0.0181259,-0.0119651,1,-0.000770201,0.0143157,0.0256983,0.00481835,-0.0266847,-0.0107505,-0.00850443,0.00117325,-0.00267348,-0.00376146,-0.00164759,0.00321289,-0.00499853,0.00603204,0.00238391,0.0146685
first_deposit_amount,-0.0684815,0.0159824,0.127102,0.0168379,-0.000770201,1,0.00689557,-0.0144703,0.0297024,-0.00429384,-0.00544079,0.0220595,-0.0116148,0.0246933,-0.00562974,-0.0172641,-0.0101865,-0.00949738,-0.0152202,-0.00368726,-0.0147879
time_horizon,-0.0494142,0.152741,0.124193,0.011693,0.0143157,0.00689557,1,0.0317281,-0.00317847,-0.027506,0.00136,0.0078829,0.0454196,0.0423575,0.00756723,0.0320635,0.0300406,0.00802698,-0.059857,-0.0122564,-0.0191912
Android,0.0322114,0.00633151,0.0290484,0.290207,0.0256983,-0.0144703,0.0317281,1,-0.193627,-0.811236,-0.0213298,-0.02382,0.00352022,-0.0283978,0.00859474,-0.00431067,0.0119561,0.0153742,0.0284848,-0.00821816,0.0196769
both,-0.0111972,-0.0151734,0.0282009,0.0593602,0.00481835,0.0297024,-0.00317847,-0.193627,1,-0.416577,-0.00227612,0.0175454,-0.00845299,0.00583668,0.00456464,0.0128097,0.0191324,0.00289626,-0.0228715,-0.0042201,0.00646001
iOS,-0.0231748,0.00317631,-0.043725,-0.304295,-0.0266847,-0.00429384,-0.027506,-0.811236,-0.416577,1,0.0211215,0.0116155,0.00177601,0.0228358,-0.0106847,-0.00364013,-0.0224819,-0.0159724,-0.0127637,0.0101304,-0.0220835


In [708]:
corr['ifchurn']

ifchurn                  1.000000
risk_tolerance          -0.125477
investment_experience   -0.042086
liquidity_needs         -0.066129
time_spent               0.032538
first_deposit_amount    -0.068482
time_horizon            -0.049414
Android                  0.032211
both                    -0.011197
iOS                     -0.023175
0                       -0.012958
adr                     -0.021366
cef                     -0.004679
etp                     -0.033072
lp                       0.019228
mlp                      0.005198
reit                    -0.004243
rlt                      0.004418
stock                    0.033013
tracking                -0.006270
wrt                      0.018492
Name: ifchurn, dtype: float64